In [12]:
import os
import numpy as np
import tensorflow as tf
import keras
import glob
import math
import cv2
import sklearn
from deep_svdd import BaseSVDD
from tensorflow.keras.layers import Input
import pandas as pd
import PIL
from PIL import Image
import MVTecAD_loaddata as mv
from sklearn.model_selection import GridSearchCV
from efficientnet.tfkeras import EfficientNetB2
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.svm import OneClassSVM

In [68]:
def custom_auc(ground_truth, predictions):
    fpr, tpr, _ = roc_curve(ground_truth, predictions[:, 1], pos_label=1)    
    return auc(fpr, tpr)
def data_load(file_name):
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = getattr(mv, file_name)()
    return X_train, y_tr, X_test, y_ts

def model_feature(X_train, X_test):
    f_tr = X_train.reshape(X_train.shape[0],-1)
    f_ts = X_test.reshape(X_test.shape[0],-1)
    return f_tr, f_ts

def svdd_tuning(train_data, test_data, train_label, test_label):
    tuned_parameters = [
#         {'C': [1,2**-2, 2**-3,2**-4,2**-1], 'kernel': ['linear']},
        {'C': [1,2**-2, 2**-3,2**-1], 'kernel': ['poly'], 'degree': [2,3,4]},
#         {'C': [1,2**-2, 2**-3,2**-4,2**-1], 'kernel': ['rbf'], 'gamma': [10**-4, 10**-5, 10**-6]}
    ]
    score = 'accuracy'
    clf = GridSearchCV(
        BaseSVDD(display='on'),
        tuned_parameters,
        cv=2,
        scoring=score)
    clf.fit(train_data, train_label)
    print(clf.cv_results_)
    svdd = BaseSVDD(C=clf.best_params_['C'], kernel='poly', degree=clf.best_params_['degree'])
    svdd.fit(train_data)
    pred = svdd.predict(test_data, test_label)
    dist = svdd.get_distance(test_data)
    dist = np.asarray(dist)
    a = np.where(test_label < 0, 1, 0)
    r1 = roc_auc_score(a, dist)
    r2 = f1_score(test_label, pred)
    r3 = accuracy_score(test_label, pred)
    print('roc_auc : ',r1)
    print('f1_score : ', r2)
    print('accuracy_score : ', r3)
    print('best_params : ', clf.best_params_)
    result = [r1, r2, r3, clf.best_params_['C'], clf.best_params_['degree']]
    return result
# my_auc = make_scorer(custom_auc, greater_is_better=True, needs_proba=True)
# def svdd_tuning(X, x_test, y, y_test):
#     tuned_parameters = [
#         {'nu': [2**-5, 2**-6,2**-7,2**-8], 'kernel': ['linear']},
#         {'nu': [2**-5, 2**-6,2**-7,2**-8], 'kernel': ['poly'], 'degree': [2,3,4,5,6,7,8]},
#         {'nu': [2**-5, 2**-6,2**-7,2**-8], 'kernel': ['rbf'], 'gamma': [10**-4, 10**-5, 10**-6]},
#     ]
#     score = 'accuracy'
#     clf = GridSearchCV(
#         OneClassSVM(),
#         tuned_parameters,
#         cv=5,
#         scoring=score)
#     clf.fit(X, y)
#     a = clf.decision_function(x_test)
#     b = clf.predict(x_test)
#     r1 = roc_auc_score(y_test,a)
#     r2 = f1_score(y_test,b)
#     r3 = accuracy_score(y_test,b)
#     print('roc_auc : ',r1)
#     print('f1_score : ', r2)
#     print('accuracy_score : ', r3)
#     print('best_params : ', clf.best_params_)
#     result = [r1,r2,r3,clf.best_params_['nu'],clf.best_params_['kernel'],clf.best_params_]
#     print(clf.cv_results_)
#     return result

res = []

In [69]:
X_train, Y_train, X_test, Y_test = data_load('bottle')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 0.7672 seconds
kernel function      = poly
iterations           = 10
radius               = 0.1311
number of samples    = 104
number of features   = 196608
number of SVs        = 9
ratio of SVs         = 8.6538 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4033 seconds
number of samples    = 105
number of alarm      = 105




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.7693 seconds
kernel function      = poly
iterations           = 9
radius               = nan
number of samples    = 105
number of features   = 196608
number of SVs        = 10
ratio of SVs         = 9.5238 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.3777 seconds
number of samples    = 104
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.7726 seconds
kernel function      = poly
iterations           = 9
radius               = nan
number of samples    = 104
number of features   = 196608
number of SVs        = 10
ratio of SVs         = 9.6154 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4347 seconds
number of samples    = 105
number of alarm      = 0




*** Fitting of the SVDD model is completed. ***

running time         = 0.6476 seconds
kernel function      = poly
iterations           = 8
radius               = 0.6341
number of samples    = 105
number of features   = 196608
number of SVs        = 12
ratio of SVs         = 11.4286 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.3762 seconds
number of samples    = 104
number of alarm      = 104




...



*** Fitting of the SVDD model is completed. ***

running time         = 0.6453 seconds
kernel function      = poly
iterations           = 8
radius               = nan
number of samples    = 105
number of features   = 196608
number of SVs        = 10
ratio of SVs         = 9.5238 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.3787 seconds
number of samples    = 104
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.4910 seconds
kernel function      = poly
iterations           = 8
radius               = nan
number of samples    = 209
number of features   = 196608
number of SVs        = 11
ratio of SVs         = 5.2632 %
accuracy             = 100.0000 %


{'mean_fit_time': array([0.97885227, 0.85204768, 0.82819426, 0.80500364, 0.79951036,
       0.8044889 , 0.80513299, 0.80761051, 0.80338955, 0.79130447,
       0.83105612, 0.81192648]), 'std_fit_time': array([0.02052402, 0.06240392, 0.00768459, 0.0122261 , 0.01293433,
       0.01694584, 0.00181115, 0.0087626 , 0.01417446, 0.00545943,
       0.03303385, 0.00758898]), 'mean_score_time': array([0.39047217, 0.40547013, 0.3580184 , 0.39400959, 0.37018108,
       0.37944949, 0.39607668, 0.37559032, 0.40121293, 0.37017369,
       0.36688566, 0.40181947]), 'std_score_time': array([0.01280236, 0.02926064, 0.00563169, 0.02926183, 0.00692749,
       0.00100935, 0.01555037, 0.00951195

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.5286 seconds
kernel function      = poly
iterations           = 8
radius               = nan
number of samples    = 209
number of features   = 196608
number of SVs        = 11
ratio of SVs         = 5.2632 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.5377 seconds
number of samples    = 83
number of alarm      = 0
accuracy             = 24.0964 %


roc_auc :  0.8428571428571429
f1_score :  0.38834951456310685
accuracy_score :  0.24096385542168675
best_params :  {'C': 1, 'degree': 4, 'kernel': 'poly'}


In [70]:
X_train, Y_train, X_test, Y_test = data_load('cable')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 0.8952 seconds
kernel function      = poly
iterations           = 8
radius               = 1.7369
number of samples    = 112
number of features   = 196608
number of SVs        = 25
ratio of SVs         = 22.3214 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4583 seconds
number of samples    = 112
number of alarm      = 112




*** Fitting of the SVDD model is completed. ***

running time         = 0.7017 seconds
kernel function      = poly
iterations           = 8
radius               = 1.7645
number of samples    = 112
number of features   = 196608
number of SVs        = 26
ratio of SVs         = 23.2143 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4105 seconds
number of samples    = 112
number of alarm      = 112




*** Fitting of the SVDD model is completed. ***

run



*** Prediction of the provided data is completed. ***

running time         = 0.3873 seconds
number of samples    = 112
number of alarm      = 112




*** Fitting of the SVDD model is completed. ***

running time         = 0.7394 seconds
kernel function      = poly
iterations           = 8
radius               = 1.7221
number of samples    = 112
number of features   = 196608
number of SVs        = 24
ratio of SVs         = 21.4286 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4035 seconds
number of samples    = 112
number of alarm      = 112




*** Fitting of the SVDD model is completed. ***

running time         = 0.7106 seconds
kernel function      = poly
iterations           = 8
radius               = 1.7645
number of samples    = 112
number of features   = 196608
number of SVs        = 26
ratio of SVs         = 23.2143 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. **

In [71]:
X_train, Y_train, X_test, Y_test = data_load('capsule')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 0.9231 seconds
kernel function      = poly
iterations           = 9
radius               = 1.8737
number of samples    = 109
number of features   = 196608
number of SVs        = 6
ratio of SVs         = 5.5046 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4934 seconds
number of samples    = 110
number of alarm      = 3




*** Fitting of the SVDD model is completed. ***

running time         = 0.8433 seconds
kernel function      = poly
iterations           = 8
radius               = 3.3874
number of samples    = 110
number of features   = 196608
number of SVs        = 5
ratio of SVs         = 4.5455 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4525 seconds
number of samples    = 109
number of alarm      = 0




*** Fitting of the SVDD model is completed. ***

running

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.8725 seconds
kernel function      = poly
iterations           = 12
radius               = nan
number of samples    = 110
number of features   = 196608
number of SVs        = 2
ratio of SVs         = 1.8182 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4843 seconds
number of samples    = 109
number of alarm      = 0




*** Fitting of the SVDD model is completed. ***

running time         = 0.7118 seconds
kernel function      = poly
iterations           = 12
radius               = 23.5330
number of samples    = 109
number of features   = 196608
number of SVs        = 4
ratio of SVs         = 3.6697 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4452 seconds
number of samples    = 110
number of alarm      = 7




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.7821 seconds
kernel function      = poly
iterations           = 19
radius               = nan
number of samples    = 110
number of features   = 196608
number of SVs        = 2
ratio of SVs         = 1.8182 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4108 seconds
number of samples    = 109
number of alarm      = 0




*** Fitting of the SVDD model is completed. ***

running time         = 1.0657 seconds
kernel function      = poly
iterations           = 9
radius               = 1.8722
number of samples    = 109
number of features   = 196608
number of SVs        = 6
ratio of SVs         = 5.5046 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.3485 seconds
number of samples    = 110
number of alarm      = 3




*** Fitting of the SVDD model is completed. ***

running t



*** Prediction of the provided data is completed. ***

running time         = 0.4450 seconds
number of samples    = 110
number of alarm      = 38




*** Fitting of the SVDD model is completed. ***

running time         = 0.7576 seconds
kernel function      = poly
iterations           = 20
radius               = 27.0091
number of samples    = 110
number of features   = 196608
number of SVs        = 3
ratio of SVs         = 2.7273 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.3802 seconds
number of samples    = 109
number of alarm      = 0




*** Fitting of the SVDD model is completed. ***

running time         = 1.5282 seconds
kernel function      = poly
iterations           = 8
radius               = 3.0348
number of samples    = 219
number of features   = 196608
number of SVs        = 5
ratio of SVs         = 2.2831 %
accuracy             = 100.0000 %


{'mean_fit_time': array([1.06754339, 1.27001679, 0.8930

In [72]:
X_train, Y_train, X_test, Y_test = data_load('carpet')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 1.1359 seconds
kernel function      = poly
iterations           = 7
radius               = 2.9223
number of samples    = 140
number of features   = 196608
number of SVs        = 20
ratio of SVs         = 14.2857 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4867 seconds
number of samples    = 140
number of alarm      = 140




*** Fitting of the SVDD model is completed. ***

running time         = 1.0097 seconds
kernel function      = poly
iterations           = 6
radius               = 2.3593
number of samples    = 140
number of features   = 196608
number of SVs        = 15
ratio of SVs         = 10.7143 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4984 seconds
number of samples    = 140
number of alarm      = 140




*** Fitting of the SVDD model is completed. ***

run



*** Prediction of the provided data is completed. ***

running time         = 0.5007 seconds
number of samples    = 140
number of alarm      = 140




*** Fitting of the SVDD model is completed. ***

running time         = 0.9174 seconds
kernel function      = poly
iterations           = 7
radius               = 2.9223
number of samples    = 140
number of features   = 196608
number of SVs        = 20
ratio of SVs         = 14.2857 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4934 seconds
number of samples    = 140
number of alarm      = 140




*** Fitting of the SVDD model is completed. ***

running time         = 0.9291 seconds
kernel function      = poly
iterations           = 6
radius               = 2.3593
number of samples    = 140
number of features   = 196608
number of SVs        = 15
ratio of SVs         = 10.7143 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. **

In [74]:
X_train, Y_train, X_test, Y_test = data_load('grid')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.9098 seconds
kernel function      = poly
iterations           = 9
radius               = nan
number of samples    = 132
number of features   = 196608
number of SVs        = 17
ratio of SVs         = 12.8788 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4751 seconds
number of samples    = 132
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.9436 seconds
kernel function      = poly
iterations           = 9
radius               = nan
number of samples    = 132
number of features   = 196608
number of SVs        = 15
ratio of SVs         = 11.3636 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4504 seconds
number of samples    = 132
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))


...



*** Fitting of the SVDD model is completed. ***

running time         = 0.8678 seconds
kernel function      = poly
iterations           = 18
radius               = nan
number of samples    = 132
number of features   = 196608
number of SVs        = 11
ratio of SVs         = 8.3333 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4833 seconds
number of samples    = 132
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.9274 seconds
kernel function      = poly
iterations           = 10
radius               = nan
number of samples    = 264
number of features   = 196608
number of SVs        = 15
ratio of SVs         = 5.6818 %
accuracy             = 100.0000 %


{'mean_fit_time': array([1.16048002, 1.04435658, 1.08703327, 1.03891635, 1.07663882,
       1.07720709, 1.01123798, 1.03319073, 1.05738604, 1.00936508,
       1.10672414, 1.05873084]), 'std_fit_time': array([0.03205752, 0.00337791, 0.03608751, 0.02860093, 0.01327765,
       0.00246096, 0.00383937, 0.01286721, 0.01942575, 0.00460911,
       0.06101167, 0.01701307]), 'mean_score_time': array([0.46449387, 0.44650233, 0.44144714, 0.42706084, 0.44790792,
       0.4582262 , 0.44721675, 0.45331275, 0.45100546, 0.45811689,
       0.4629606 , 0.46309531]), 'std_score_time': array([0.0141331 , 0.00354588, 0.01606047, 0.00110149, 0.02553558,
       0.01579046, 0.02077603, 0.0128823

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.9171 seconds
kernel function      = poly
iterations           = 10
radius               = nan
number of samples    = 264
number of features   = 196608
number of SVs        = 15
ratio of SVs         = 5.6818 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.5076 seconds
number of samples    = 78
number of alarm      = 0
accuracy             = 26.9231 %


roc_auc :  0.25898078529657476
f1_score :  0.42424242424242425
accuracy_score :  0.2692307692307692
best_params :  {'C': 1, 'degree': 2, 'kernel': 'poly'}


In [75]:
X_train, Y_train, X_test, Y_test = data_load('hazelnut')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.5890 seconds
kernel function      = poly
iterations           = 8
radius               = nan
number of samples    = 195
number of features   = 196608
number of SVs        = 4
ratio of SVs         = 2.0513 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.9025 seconds
number of samples    = 196
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.6219 seconds
kernel function      = poly
iterations           = 8
radius               = nan
number of samples    = 196
number of features   = 196608
number of SVs        = 4
ratio of SVs         = 2.0408 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.9065 seconds
number of samples    = 195
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))


...



*** Fitting of the SVDD model is completed. ***

running time         = 1.6994 seconds
kernel function      = poly
iterations           = 21
radius               = nan
number of samples    = 196
number of features   = 196608
number of SVs        = 3
ratio of SVs         = 1.5306 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.8459 seconds
number of samples    = 195
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 3.3206 seconds
kernel function      = poly
iterations           = 10
radius               = nan
number of samples    = 391
number of features   = 196608
number of SVs        = 4
ratio of SVs         = 1.0230 %
accuracy             = 100.0000 %


{'mean_fit_time': array([2.00863671, 2.00831842, 2.09176588, 1.9407773 , 2.04639864,
       2.12889707, 1.93338823, 1.97517431, 1.98824656, 1.95087576,
       1.97572243, 1.98994827]), 'std_fit_time': array([0.04876852, 0.02489805, 0.02849865, 0.00701952, 0.1041975 ,
       0.07410324, 0.03670502, 0.00694191, 0.02046931, 0.03698874,
       0.04161823, 0.02398849]), 'mean_score_time': array([0.90451074, 0.91965377, 0.91522813, 0.91510808, 0.90775537,
       0.88201201, 0.83136749, 0.8536967 , 0.83759713, 0.7799046 ,
       0.79053962, 0.8388536 ]), 'std_score_time': array([0.00196362, 0.00623858, 0.01220608, 0.01532924, 0.01314473,
       0.00810659, 0.00231886, 0.00138319

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 3.5372 seconds
kernel function      = poly
iterations           = 10
radius               = nan
number of samples    = 391
number of features   = 196608
number of SVs        = 4
ratio of SVs         = 1.0230 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.8911 seconds
number of samples    = 110
number of alarm      = 0
accuracy             = 36.3636 %


roc_auc :  0.6585714285714286
f1_score :  0.5333333333333333
accuracy_score :  0.36363636363636365
best_params :  {'C': 1, 'degree': 2, 'kernel': 'poly'}


In [76]:
X_train, Y_train, X_test, Y_test = data_load('leather')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 0.8609 seconds
kernel function      = poly
iterations           = 7
radius               = 0.7691
number of samples    = 122
number of features   = 196608
number of SVs        = 5
ratio of SVs         = 4.0984 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4938 seconds
number of samples    = 123
number of alarm      = 123




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.7818 seconds
kernel function      = poly
iterations           = 7
radius               = nan
number of samples    = 123
number of features   = 196608
number of SVs        = 12
ratio of SVs         = 9.7561 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.5328 seconds
number of samples    = 122
number of alarm      = 0




*** Fitting of the SVDD model is completed. ***

running time         = 0.7799 seconds
kernel function      = poly
iterations           = 13
radius               = 20.3756
number of samples    = 122
number of features   = 196608
number of SVs        = 4
ratio of SVs         = 3.2787 %
accuracy             = 70.4918 %




*** Prediction of the provided data is completed. ***

running time         = 0.4351 seconds
number of samples    = 123
number of alarm      = 6




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.8144 seconds
kernel function      = poly
iterations           = 13
radius               = nan
number of samples    = 123
number of features   = 196608
number of SVs        = 10
ratio of SVs         = 8.1301 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4286 seconds
number of samples    = 122
number of alarm      = 0




*** Fitting of the SVDD model is completed. ***

running time         = 0.8044 seconds
kernel function      = poly
iterations           = 16
radius               = 91.5788
number of samples    = 122
number of features   = 196608
number of SVs        = 4
ratio of SVs         = 3.2787 %
accuracy             = 35.2459 %




*** Prediction of the provided data is completed. ***

running time         = 0.4178 seconds
number of samples    = 123
number of alarm      = 17




...



*** Fitting of the SVDD model is completed. ***

running time         = 0.8130 seconds
kernel function      = poly
iterations           = 17
radius               = nan
number of samples    = 123
number of features   = 196608
number of SVs        = 8
ratio of SVs         = 6.5041 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4275 seconds
number of samples    = 122
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.6795 seconds
kernel function      = poly
iterations           = 8
radius               = nan
number of samples    = 245
number of features   = 196608
number of SVs        = 11
ratio of SVs         = 4.4898 %
accuracy             = 100.0000 %


{'mean_fit_time': array([0.99744976, 0.96451426, 0.98174965, 0.93915534, 0.97848058,
       0.98969996, 0.9315213 , 0.98573411, 1.03597999, 0.94502008,
       0.94945168, 0.97331953]), 'std_fit_time': array([0.0440625 , 0.0177331 , 0.00441849, 0.0026083 , 0.00899363,
       0.00296319, 0.0067786 , 0.02705348, 0.00916743, 0.00468671,
       0.00777626, 0.00257683]), 'mean_score_time': array([0.51444674, 0.4318589 , 0.42256761, 0.42016196, 0.42957258,
       0.44615972, 0.41892445, 0.45868933, 0.43740964, 0.43887222,
       0.42527378, 0.42690086]), 'std_score_time': array([0.02064371, 0.00324142, 0.00481009, 0.00542998, 0.00334668,
       0.011778  , 0.00701201, 0.03165185

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.6525 seconds
kernel function      = poly
iterations           = 8
radius               = nan
number of samples    = 245
number of features   = 196608
number of SVs        = 11
ratio of SVs         = 4.4898 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.5715 seconds
number of samples    = 124
number of alarm      = 0
accuracy             = 25.8065 %


roc_auc :  0.3913043478260869
f1_score :  0.41025641025641024
accuracy_score :  0.25806451612903225
best_params :  {'C': 0.25, 'degree': 2, 'kernel': 'poly'}


In [77]:
X_train, Y_train, X_test, Y_test = data_load('metal_nut')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 0.7735 seconds
kernel function      = poly
iterations           = 8
radius               = 0.7438
number of samples    = 110
number of features   = 196608
number of SVs        = 15
ratio of SVs         = 13.6364 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.3952 seconds
number of samples    = 110
number of alarm      = 110




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.6657 seconds
kernel function      = poly
iterations           = 8
radius               = nan
number of samples    = 110
number of features   = 196608
number of SVs        = 11
ratio of SVs         = 10.0000 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.3839 seconds
number of samples    = 110
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.7000 seconds
kernel function      = poly
iterations           = 8
radius               = nan
number of samples    = 110
number of features   = 196608
number of SVs        = 12
ratio of SVs         = 10.9091 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.3847 seconds
number of samples    = 110
number of alarm      = 0




...



*** Fitting of the SVDD model is completed. ***

running time         = 0.7162 seconds
kernel function      = poly
iterations           = 14
radius               = nan
number of samples    = 110
number of features   = 196608
number of SVs        = 7
ratio of SVs         = 6.3636 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.3783 seconds
number of samples    = 110
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.5706 seconds
kernel function      = poly
iterations           = 9
radius               = nan
number of samples    = 220
number of features   = 196608
number of SVs        = 10
ratio of SVs         = 4.5455 %
accuracy             = 100.0000 %


{'mean_fit_time': array([0.8688457 , 0.85016727, 0.84539974, 0.81213892, 0.82549572,
       0.86854649, 0.82758307, 0.82684135, 0.8316505 , 0.82244587,
       0.81760788, 0.84646749]), 'std_fit_time': array([0.04602861, 0.00690413, 0.00420868, 0.00682747, 0.00083733,
       0.0274837 , 0.00965285, 0.01216626, 0.00873899, 0.00278425,
       0.00430202, 0.01159501]), 'mean_score_time': array([0.38958037, 0.38808334, 0.3840344 , 0.3746258 , 0.37781191,
       0.38218951, 0.38280463, 0.3777529 , 0.37767982, 0.37822258,
       0.41044271, 0.38171697]), 'std_score_time': array([0.0056361 , 0.00334394, 0.00751972, 0.00426996, 0.0004375 ,
       0.00220895, 0.01308393, 0.00068533

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.5233 seconds
kernel function      = poly
iterations           = 9
radius               = nan
number of samples    = 220
number of features   = 196608
number of SVs        = 10
ratio of SVs         = 4.5455 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.5970 seconds
number of samples    = 115
number of alarm      = 0
accuracy             = 19.1304 %


roc_auc :  0.43841642228739003
f1_score :  0.32116788321167883
accuracy_score :  0.19130434782608696
best_params :  {'C': 1, 'degree': 3, 'kernel': 'poly'}


In [78]:
X_train, Y_train, X_test, Y_test = data_load('pill')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 0.9103 seconds
kernel function      = poly
iterations           = 9
radius               = 0.2929
number of samples    = 133
number of features   = 196608
number of SVs        = 15
ratio of SVs         = 11.2782 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4397 seconds
number of samples    = 134
number of alarm      = 3




*** Fitting of the SVDD model is completed. ***

running time         = 0.8420 seconds
kernel function      = poly
iterations           = 11
radius               = 0.1232
number of samples    = 134
number of features   = 196608
number of SVs        = 16
ratio of SVs         = 11.9403 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4341 seconds
number of samples    = 133
number of alarm      = 133




*** Fitting of the SVDD model is completed. ***

ru



*** Prediction of the provided data is completed. ***

running time         = 0.4492 seconds
number of samples    = 133
number of alarm      = 133




*** Fitting of the SVDD model is completed. ***

running time         = 0.8420 seconds
kernel function      = poly
iterations           = 10
radius               = 0.2796
number of samples    = 133
number of features   = 196608
number of SVs        = 14
ratio of SVs         = 10.5263 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4302 seconds
number of samples    = 134
number of alarm      = 5




*** Fitting of the SVDD model is completed. ***

running time         = 0.8585 seconds
kernel function      = poly
iterations           = 11
radius               = 0.1232
number of samples    = 134
number of features   = 196608
number of SVs        = 16
ratio of SVs         = 11.9403 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. 

In [79]:
X_train, Y_train, X_test, Y_test = data_load('screw')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.1278 seconds
kernel function      = poly
iterations           = 10
radius               = nan
number of samples    = 160
number of features   = 196608
number of SVs        = 13
ratio of SVs         = 8.1250 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.5299 seconds
number of samples    = 160
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.0958 seconds
kernel function      = poly
iterations           = 9
radius               = nan
number of samples    = 160
number of features   = 196608
number of SVs        = 14
ratio of SVs         = 8.7500 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.5694 seconds
number of samples    = 160
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))


...



*** Fitting of the SVDD model is completed. ***

running time         = 1.1087 seconds
kernel function      = poly
iterations           = 23
radius               = nan
number of samples    = 160
number of features   = 196608
number of SVs        = 9
ratio of SVs         = 5.6250 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.5591 seconds
number of samples    = 160
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 2.4830 seconds
kernel function      = poly
iterations           = 10
radius               = nan
number of samples    = 320
number of features   = 196608
number of SVs        = 13
ratio of SVs         = 4.0625 %
accuracy             = 100.0000 %


{'mean_fit_time': array([1.37668931, 1.30092204, 1.33274829, 1.25072634, 1.28704238,
       1.35058475, 1.25331724, 1.33312941, 1.37604558, 1.26556039,
       1.29141986, 1.33422518]), 'std_fit_time': array([0.05017459, 0.00369561, 0.01238   , 0.00868928, 0.00411916,
       0.01140046, 0.00658143, 0.02534699, 0.01606691, 0.00879693,
       0.00058067, 0.00254297]), 'mean_score_time': array([0.54962695, 0.5647217 , 0.56640983, 0.56417835, 0.57673895,
       0.56878996, 0.56039178, 0.56901824, 0.55986369, 0.56084597,
       0.56077945, 0.575562  ]), 'std_score_time': array([1.97454691e-02, 5.44989109e-03, 1.97815895e-03, 3.64196301e-03,
       2.16364861e-04, 5.53226471e-0

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 2.4563 seconds
kernel function      = poly
iterations           = 10
radius               = nan
number of samples    = 320
number of features   = 196608
number of SVs        = 13
ratio of SVs         = 4.0625 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.7561 seconds
number of samples    = 160
number of alarm      = 0
accuracy             = 25.6250 %


roc_auc :  0.011682721869235501
f1_score :  0.40796019900497504
accuracy_score :  0.25625
best_params :  {'C': 1, 'degree': 2, 'kernel': 'poly'}


In [80]:
X_train, Y_train, X_test, Y_test = data_load('tile')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 0.7860 seconds
kernel function      = poly
iterations           = 8
radius               = 1.6640
number of samples    = 115
number of features   = 196608
number of SVs        = 12
ratio of SVs         = 10.4348 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4182 seconds
number of samples    = 115
number of alarm      = 115




*** Fitting of the SVDD model is completed. ***

running time         = 0.7353 seconds
kernel function      = poly
iterations           = 8
radius               = 2.0810
number of samples    = 115
number of features   = 196608
number of SVs        = 11
ratio of SVs         = 9.5652 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.3979 seconds
number of samples    = 115
number of alarm      = 115




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.7462 seconds
kernel function      = poly
iterations           = 14
radius               = nan
number of samples    = 115
number of features   = 196608
number of SVs        = 9
ratio of SVs         = 7.8261 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.3917 seconds
number of samples    = 115
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.7192 seconds
kernel function      = poly
iterations           = 14
radius               = nan
number of samples    = 115
number of features   = 196608
number of SVs        = 7
ratio of SVs         = 6.0870 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4030 seconds
number of samples    = 115
number of alarm      = 0




...



*** Fitting of the SVDD model is completed. ***

running time         = 0.7263 seconds
kernel function      = poly
iterations           = 17
radius               = nan
number of samples    = 115
number of features   = 196608
number of SVs        = 6
ratio of SVs         = 5.2174 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4142 seconds
number of samples    = 115
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.7267 seconds
kernel function      = poly
iterations           = 16
radius               = nan
number of samples    = 230
number of features   = 196608
number of SVs        = 8
ratio of SVs         = 3.4783 %
accuracy             = 100.0000 %


{'mean_fit_time': array([0.92570627, 0.88800335, 0.89472592, 0.8410393 , 0.88602352,
       0.87449086, 0.85380292, 0.88698065, 0.90689588, 0.85015965,
       0.90091133, 0.88966179]), 'std_fit_time': array([3.24262381e-02, 1.48763657e-02, 4.19580936e-03, 5.61630726e-03,
       5.37562370e-03, 2.80916691e-03, 3.44705582e-03, 1.12878084e-02,
       2.25217342e-02, 9.56058502e-05, 1.11708641e-02, 1.74641609e-03]), 'mean_score_time': array([0.40804446, 0.39734876, 0.40605664, 0.39756334, 0.40048003,
       0.40352464, 0.396716  , 0.43151164, 0.4113574 , 0.40173924,
       0.39451671, 0.41240644]), 'std_score_time': array([1.01584196e-02, 5.62846661e-03, 4.54807281e-03, 3.010

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.7680 seconds
kernel function      = poly
iterations           = 16
radius               = nan
number of samples    = 230
number of features   = 196608
number of SVs        = 8
ratio of SVs         = 3.4783 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.5781 seconds
number of samples    = 117
number of alarm      = 0
accuracy             = 28.2051 %


roc_auc :  0.6843434343434344
f1_score :  0.44000000000000006
accuracy_score :  0.28205128205128205
best_params :  {'C': 1, 'degree': 3, 'kernel': 'poly'}


In [81]:
X_train, Y_train, X_test, Y_test = data_load('toothbrush')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 0.2198 seconds
kernel function      = poly
iterations           = 7
radius               = 0.5643
number of samples    = 30
number of features   = 196608
number of SVs        = 12
ratio of SVs         = 40.0000 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.0943 seconds
number of samples    = 30
number of alarm      = 20




*** Fitting of the SVDD model is completed. ***

running time         = 0.2061 seconds
kernel function      = poly
iterations           = 8
radius               = 0.5844
number of samples    = 30
number of features   = 196608
number of SVs        = 10
ratio of SVs         = 33.3333 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.0942 seconds
number of samples    = 30
number of alarm      = 23




*** Fitting of the SVDD model is completed. ***

runni



*** Fitting of the SVDD model is completed. ***

running time         = 0.2262 seconds
kernel function      = poly
iterations           = 7
radius               = 0.6424
number of samples    = 30
number of features   = 196608
number of SVs        = 11
ratio of SVs         = 36.6667 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.0939 seconds
number of samples    = 30
number of alarm      = 20




*** Fitting of the SVDD model is completed. ***

running time         = 0.2217 seconds
kernel function      = poly
iterations           = 8
radius               = 0.6329
number of samples    = 30
number of features   = 196608
number of SVs        = 9
ratio of SVs         = 30.0000 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.0939 seconds
number of samples    = 30
number of alarm      = 20




*** Fitting of the SVDD model is completed. ***

runnin

In [82]:
X_train, Y_train, X_test, Y_test = data_load('transistor')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 0.7549 seconds
kernel function      = poly
iterations           = 7
radius               = 1.7178
number of samples    = 106
number of features   = 196608
number of SVs        = 10
ratio of SVs         = 9.4340 %
accuracy             = 33.9623 %




*** Prediction of the provided data is completed. ***

running time         = 0.3557 seconds
number of samples    = 107
number of alarm      = 90




*** Fitting of the SVDD model is completed. ***

running time         = 0.6941 seconds
kernel function      = poly
iterations           = 8
radius               = 1.6483
number of samples    = 107
number of features   = 196608
number of SVs        = 14
ratio of SVs         = 13.0841 %
accuracy             = 23.3645 %




*** Prediction of the provided data is completed. ***

running time         = 0.3970 seconds
number of samples    = 106
number of alarm      = 78




*** Fitting of the SVDD model is completed. ***

runn



*** Prediction of the provided data is completed. ***

running time         = 0.4104 seconds
number of samples    = 106
number of alarm      = 102




*** Fitting of the SVDD model is completed. ***

running time         = 0.7020 seconds
kernel function      = poly
iterations           = 7
radius               = 1.7178
number of samples    = 106
number of features   = 196608
number of SVs        = 10
ratio of SVs         = 9.4340 %
accuracy             = 33.9623 %




*** Prediction of the provided data is completed. ***

running time         = 0.3643 seconds
number of samples    = 107
number of alarm      = 90




*** Fitting of the SVDD model is completed. ***

running time         = 0.6621 seconds
kernel function      = poly
iterations           = 8
radius               = 1.6483
number of samples    = 107
number of features   = 196608
number of SVs        = 14
ratio of SVs         = 13.0841 %
accuracy             = 23.3645 %




*** Prediction of the provided data is completed. **

In [83]:
X_train, Y_train, X_test, Y_test = data_load('wood')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 0.8332 seconds
kernel function      = poly
iterations           = 9
radius               = 5.1428
number of samples    = 123
number of features   = 196608
number of SVs        = 5
ratio of SVs         = 4.0650 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4410 seconds
number of samples    = 124
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.8386 seconds
kernel function      = poly
iterations           = 8
radius               = nan
number of samples    = 124
number of features   = 196608
number of SVs        = 7
ratio of SVs         = 5.6452 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4571 seconds
number of samples    = 123
number of alarm      = 0




*** Fitting of the SVDD model is completed. ***

running time         = 0.9437 seconds
kernel function      = poly
iterations           = 14
radius               = 18.2545
number of samples    = 123
number of features   = 196608
number of SVs        = 4
ratio of SVs         = 3.2520 %
accuracy             = 86.1789 %




*** Prediction of the provided data is completed. ***

running time         = 0.4752 seconds
number of samples    = 124
number of alarm      = 12




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 0.8418 seconds
kernel function      = poly
iterations           = 15
radius               = nan
number of samples    = 124
number of features   = 196608
number of SVs        = 3
ratio of SVs         = 2.4194 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4267 seconds
number of samples    = 123
number of alarm      = 0




...



*** Fitting of the SVDD model is completed. ***

running time         = 0.8131 seconds
kernel function      = poly
iterations           = 18
radius               = nan
number of samples    = 124
number of features   = 196608
number of SVs        = 3
ratio of SVs         = 2.4194 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4280 seconds
number of samples    = 123
number of alarm      = 0




C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.6673 seconds
kernel function      = poly
iterations           = 9
radius               = nan
number of samples    = 247
number of features   = 196608
number of SVs        = 2
ratio of SVs         = 0.8097 %
accuracy             = 100.0000 %


{'mean_fit_time': array([1.02459896, 1.06770015, 0.98873568, 0.97636223, 1.01330483,
       0.987481  , 0.94645131, 0.97109544, 1.00727224, 0.9597801 ,
       0.98518896, 0.98030519]), 'std_fit_time': array([0.01302946, 0.05207467, 0.0043087 , 0.0246501 , 0.02183044,
       0.01501119, 0.00032032, 0.00041282, 0.03251219, 0.00258958,
       0.00512218, 0.01007104]), 'mean_score_time': array([0.44902635, 0.45095694, 0.44680667, 0.43433774, 0.41394007,
       0.43169522, 0.41419375, 0.421821  , 0.42118251, 0.41808546,
       0.42605996, 0.41918731]), 'std_score_time': array([0.00802994, 0.02427351, 0.02024484, 0.01081145, 0.00276029,
       0.01363468, 0.00301254, 0.00690329,

C:\Users\Shuta Tsuchio\M_learning\SVDD\deep_svdd.py:269: RuntimeWarning: invalid value encountered in sqrt
  self.radius = np.sqrt(np.mean(np.diag(K)) + self.offset+np.mean(tmp_[self.boundary_indices, 0]))




*** Fitting of the SVDD model is completed. ***

running time         = 1.7272 seconds
kernel function      = poly
iterations           = 9
radius               = nan
number of samples    = 247
number of features   = 196608
number of SVs        = 2
ratio of SVs         = 0.8097 %
accuracy             = 100.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.5343 seconds
number of samples    = 79
number of alarm      = 0
accuracy             = 24.0506 %


roc_auc :  0.9263157894736842
f1_score :  0.38775510204081637
accuracy_score :  0.24050632911392406
best_params :  {'C': 1, 'degree': 2, 'kernel': 'poly'}


In [84]:
X_train, Y_train, X_test, Y_test = data_load('zipper')
X_tr, X_ts = model_feature(X_train, X_test)
result = svdd_tuning(X_tr, X_ts, Y_train, Y_test)
res.append(result)



*** Fitting of the SVDD model is completed. ***

running time         = 0.8915 seconds
kernel function      = poly
iterations           = 10
radius               = 0.7923
number of samples    = 120
number of features   = 196608
number of SVs        = 9
ratio of SVs         = 7.5000 %
accuracy             = 57.5000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4887 seconds
number of samples    = 120
number of alarm      = 60




*** Fitting of the SVDD model is completed. ***

running time         = 0.8282 seconds
kernel function      = poly
iterations           = 8
radius               = 0.6953
number of samples    = 120
number of features   = 196608
number of SVs        = 8
ratio of SVs         = 6.6667 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4710 seconds
number of samples    = 120
number of alarm      = 120




*** Fitting of the SVDD model is completed. ***

runnin



*** Prediction of the provided data is completed. ***

running time         = 0.4312 seconds
number of samples    = 120
number of alarm      = 120




*** Fitting of the SVDD model is completed. ***

running time         = 0.7630 seconds
kernel function      = poly
iterations           = 9
radius               = 0.7923
number of samples    = 120
number of features   = 196608
number of SVs        = 9
ratio of SVs         = 7.5000 %
accuracy             = 57.5000 %




*** Prediction of the provided data is completed. ***

running time         = 0.4260 seconds
number of samples    = 120
number of alarm      = 60




*** Fitting of the SVDD model is completed. ***

running time         = 0.7603 seconds
kernel function      = poly
iterations           = 8
radius               = 0.6953
number of samples    = 120
number of features   = 196608
number of SVs        = 8
ratio of SVs         = 6.6667 %
accuracy             = 0.0000 %




*** Prediction of the provided data is completed. ***

r

In [85]:
df = pd.DataFrame(res)
df.to_csv('SVDD.csv')